<a href="https://colab.research.google.com/github/Neerajk4/emma-ml/blob/master/code/notebooks/classification2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets;

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00


In [3]:
!pip install transformers;
!pip install accelerate;

In [12]:
import datasets
from datasets import load_dataset

social_ds_full = load_dataset("Yelp/yelp_review_full")
#social_ds = load_dataset("Yelp/yelp_review_full", split="train[:10%] + test[-1%]")
yelp_train = load_dataset("Yelp/yelp_review_full", split="train[:1%]")
yelp_test = load_dataset("Yelp/yelp_review_full", split="test[:1%]")

In [13]:
label_list = [l['label'] for l in yelp_train]
stuff = [0,1,2,3,4]

for s in stuff:
  print(s)
  print(label_list.count(s))

0
1132
1
1429
2
1483
3
1327
4
1129


In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [15]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [24]:
tokenized_train = yelp_train.map(preprocess_function, batched=True)
tokenized_test = yelp_test.map(preprocess_function, batched=True)

Map:   0%|          | 0/6500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [16]:
tokenized_yelp = social_ds_full.map(preprocess_function, batched=True)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [17]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [18]:
import evaluate

accuracy = evaluate.load("accuracy")

In [19]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [20]:
id2label = {0: "0", 1: "1", 2: "2", 3: "3", 4: "4"}
label2id = {"0": 0, "1": 1, "2": 2, "3": 3, "4": 4}

In [22]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

#model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", num_labels=5, id2label=id2label, label2id=label2id)
model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", num_labels=5)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.003514,0.546000
2,1.094400,0.949691,0.582000


TrainOutput(global_step=814, training_loss=1.001493299329603, metrics={'train_runtime': 619.3964, 'train_samples_per_second': 20.988, 'train_steps_per_second': 1.314, 'total_flos': 1509459828853920.0, 'train_loss': 1.001493299329603, 'epoch': 2.0})

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks/notebooks2')
currentDirectory = os.getcwd()
print(currentDirectory)

/content/gdrive/My Drive/Colab Notebooks/notebooks2


In [26]:
trainer.save_model("my_awesome_model")

In [32]:
text = "Restaurant was amazing.  We had three platters shrimp scallops, a creamy rigatoni with mushroom sauce and steak that was cooked perfectly.  The service was amazing.  I would recommend this place anyone, a definite must try!"

In [33]:
text2 = "Once again, I get stuck having to say that while this Cava is great, their chicken at this location is some of the worst served chicken at any restaurant I have ever been to. The only reason I'm even giving it anything more than a 1 star is that I know their vegetable bowl is great but their chicken just ruins my day."

In [29]:
text3 = "Cava is one of my favorite fast casual chains. Everything they have is delicious in terms of proteins toppings and sauces, you really can't go wrong. Now if only they would open in Arlington..."

In [37]:
text4 = "It's like a chipotle but for Mediterranean food. A good place to grab a quick bite. Portions are more generous than Chipotle"

In [38]:
text5 = "It was good but my salad was swimming in salad dressing. The rest of it, in the photo was inedible in my opinion. I forgot to ask for dressing on the side, but still feel it was a ridiculous amount of dressing."

In [41]:
text6 = "My friend brought me here for dinner and WOW it is amazing! Everything is fresh and delicious. I love that you can customize your own meal. "

In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name_or_path = "my_awesome_model" #path/to/your/model/or/name/on/hub
##device = "cpu" # or "cuda" if you have a GPU

model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

ValueError: Unrecognized configuration class <class 'transformers.models.distilbert.configuration_distilbert.DistilBertConfig'> for this kind of AutoModel: AutoModelForCausalLM.
Model type should be one of BartConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BlenderbotConfig, BlenderbotSmallConfig, BloomConfig, CamembertConfig, LlamaConfig, CodeGenConfig, CohereConfig, CpmAntConfig, CTRLConfig, Data2VecTextConfig, DbrxConfig, ElectraConfig, ErnieConfig, FalconConfig, FalconMambaConfig, FuyuConfig, GemmaConfig, Gemma2Config, GitConfig, GlmConfig, GPT2Config, GPT2Config, GPTBigCodeConfig, GPTNeoConfig, GPTNeoXConfig, GPTNeoXJapaneseConfig, GPTJConfig, GraniteConfig, GraniteMoeConfig, JambaConfig, JetMoeConfig, LlamaConfig, MambaConfig, Mamba2Config, MarianConfig, MBartConfig, MegaConfig, MegatronBertConfig, MistralConfig, MixtralConfig, MllamaConfig, MoshiConfig, MptConfig, MusicgenConfig, MusicgenMelodyConfig, MvpConfig, NemotronConfig, OlmoCo

In [28]:
from transformers import pipeline
classifier = pipeline('text-classification', model="my_awesome_model")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [42]:
classifier(text6)

[{'label': 'LABEL_4', 'score': 0.793853759765625}]